In [36]:
import os
import pandas as pd
import numpy as np

In [37]:
os.getcwd()

'/Users/caseyh/Desktop/footyfootyfooty/data_wrangling'

In [38]:
seasons = sorted(os.listdir('/Users/caseyh/Desktop/footyfootyfooty/round_results/'))[-25:-1]

In [39]:
season_data = pd.DataFrame({})
for season in seasons:
    path = '/Users/caseyh/Desktop/footyfootyfooty/round_results/'
    tmp = pd.read_csv(path + season)
    season_data = pd.concat([season_data, tmp])
season_data = season_data.reset_index(drop=True)

In [40]:
season_data

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score
0,Wed 8 Mar,MCG,41648.0,R1,2000,MEL,RCH,92.0,94.0
1,Thu 9 Mar,Marvel Stadium,43012.0,R1,2000,ESS,POR,156.0,62.0
2,Fri 10 Mar,MCG,20580.0,R1,2000,NTH,WCE,111.0,154.0
3,Sat 11 Mar,Domain Stadium,19777.0,R1,2000,FRE,GEE,107.0,129.0
4,Sat 11 Mar,AAMI Stadium,37222.0,R1,2000,ADE,WBD,108.0,131.0
...,...,...,...,...,...,...,...,...,...
4262,Fri 25 Aug 12:00pm,GMHBA Stadium,NaN,R24,2023,GEE,WBD,NaN,NaN
4263,Fri 25 Aug 12:00pm,MCG,NaN,R24,2023,ESS,COL,NaN,NaN
4264,Fri 25 Aug 12:00pm,Marvel Stadium,NaN,R24,2023,CAR,GWS,NaN,NaN
4265,Fri 25 Aug 12:00pm,Gabba,NaN,R24,2023,BRL,STK,NaN,NaN


In [41]:
# change round values to ints
season_data['round'] = season_data['round'].str[1:].astype(int)

In [42]:
teams = ['RCH', 'GEE', 'NTH', 'POR', 'GCS', 'MEL', 'GWS', 'HAW', 'STK',
       'CAR', 'BRL', 'COL', 'ADE', 'FRE', 'WBD', 'SYD', 'ESS', 'WCE']

In [43]:
# rearrange game table
output = pd.DataFrame({})
for season in season_data['year'].unique():
    for r in season_data['round'].unique():
        tmp1 = season_data[(season_data['round'] == r)&(season_data['year']==season)]
        for i,j in [('Home','Away'), ('Away', 'Home')]:
            tmp = pd.DataFrame({})

            tmp['team'] = tmp1[f'{i}']
            tmp['result'] = np.select([tmp1[f'{i} Score'] > tmp1[f'{j} Score'], tmp1[f'{i} Score'] < tmp1[f'{j} Score']], [4, 0], 2)
            tmp['pf'] = tmp1[f'{i} Score']
            tmp['pa'] = tmp1[f'{j} Score']
            tmp['round'] = r
            tmp['game played'] = 1
            tmp['year'] = season
            output = pd.concat([output, tmp])

output = output.reset_index(drop = True)

In [44]:
output

,team,result,pf,pa,round,game played,year
0,MEL,0,92.0,94.0,1,1,2000
1,ESS,4,156.0,62.0,1,1,2000
2,NTH,0,111.0,154.0,1,1,2000
3,FRE,0,107.0,129.0,1,1,2000
4,ADE,0,108.0,131.0,1,1,2000
...,...,...,...,...,...,...,...
8529,WBD,2,NaN,NaN,24,1,2023
8530,COL,2,NaN,NaN,24,1,2023
8531,GWS,2,NaN,NaN,24,1,2023
8532,STK,2,NaN,NaN,24,1,2023


In [45]:
# add in bye rounds
for season in season_data['year'].unique():
    
    for r in season_data[season_data['year']==season]['round'].unique():
        tmp = output[(output['round']==r)&(output['year']==season)]['team'].unique()
        
        for team in season_data[season_data['year']==season]['Home'].unique():
            if team not in tmp:
                new_row = pd.DataFrame({'team':team, 'result':0,'pf':0,'pa':0,'round':r,'game played':0,'year':season}, index=[0])
                output = pd.concat([new_row, output])

In [46]:
output

,team,result,pf,pa,round,game played,year
0,WBD,0,0.0,0.0,15,0,2023
0,CAR,0,0.0,0.0,15,0,2023
0,GWS,0,0.0,0.0,15,0,2023
0,POR,0,0.0,0.0,15,0,2023
0,NTH,0,0.0,0.0,15,0,2023
...,...,...,...,...,...,...,...
8529,WBD,2,NaN,NaN,24,1,2023
8530,COL,2,NaN,NaN,24,1,2023
8531,GWS,2,NaN,NaN,24,1,2023
8532,STK,2,NaN,NaN,24,1,2023


In [47]:
output = output.sort_values(['year','round']).reset_index(drop=True)
output

,team,result,pf,pa,round,game played,year
0,MEL,0,92.0,94.0,1,1,2000
1,ESS,4,156.0,62.0,1,1,2000
2,NTH,0,111.0,154.0,1,1,2000
3,FRE,0,107.0,129.0,1,1,2000
4,ADE,0,108.0,131.0,1,1,2000
...,...,...,...,...,...,...,...
8780,WBD,2,NaN,NaN,24,1,2023
8781,COL,2,NaN,NaN,24,1,2023
8782,GWS,2,NaN,NaN,24,1,2023
8783,STK,2,NaN,NaN,24,1,2023


In [48]:
round_ladder = output.groupby(['year','team'])[['result','pf','pa','game played']].cumsum()
round_ladder['team'] = output['team']
round_ladder['round'] = output['round']
round_ladder['year'] = output['year']
round_ladder['per'] = round_ladder['pf']/round_ladder['pa']

In [50]:
round_ladder = round_ladder.sort_values(['year', 'round', 'result', 'per'], ascending = [True,True,False,False])

In [51]:
round_ladder

,result,pf,pa,game played,team,round,year,per
1,4,156.0,62.0,1,ESS,1,2000,2.516129
14,4,140.0,86.0,1,COL,1,2000,1.627907
10,4,154.0,111.0,1,WCE,1,2000,1.387387
7,4,152.0,112.0,1,CAR,1,2000,1.357143
13,4,134.0,100.0,1,SYD,1,2000,1.340000
...,...,...,...,...,...,...,...,...
8779,42,NaN,NaN,23,FRE,24,2023,NaN
8768,40,NaN,NaN,23,SYD,24,2023,NaN
8770,30,NaN,NaN,23,HAW,24,2023,NaN
8769,22,NaN,NaN,23,NTH,24,2023,NaN


In [52]:
round_ladder['position'] = round_ladder.groupby(['year','round']).result.rank(method="first", ascending=False)

## Rolling Form

In [53]:
l5 = output[['year','team','result','pf','pa','game played']].groupby(['year','team']).rolling\
(5, min_periods=1, closed='left').sum().reset_index(drop=False).set_index('level_2')

In [54]:
l5['round'] = output['round']

In [55]:
l5[l5['year'] == 2023]

,year,team,result,pf,pa,game played,round
level_2,,,,,,,
8368,2023,ADE,NaN,NaN,NaN,NaN,1
8374,2023,ADE,0.0,90.0,106.0,1.0,2
8402,2023,ADE,0.0,166.0,214.0,2.0,3
8409,2023,ADE,4.0,283.0,300.0,3.0,4
8425,2023,ADE,8.0,394.0,372.0,4.0,5
...,...,...,...,...,...,...,...
8702,2023,WCE,4.0,146.0,406.0,5.0,20
8724,2023,WCE,6.0,112.0,201.0,5.0,21
8735,2023,WCE,8.0,35.0,116.0,5.0,22


## Join Data to round Data

In [56]:
main = season_data
# home team wins (1) or loses or draws (2)
main['target'] = main['Home Score'] > main['Away Score'].replace({False: 0, True: 1}, inplace=True)
main

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target
0,Wed 8 Mar,MCG,41648.0,1,2000,MEL,RCH,92.0,94.0,False
1,Thu 9 Mar,Marvel Stadium,43012.0,1,2000,ESS,POR,156.0,62.0,False
2,Fri 10 Mar,MCG,20580.0,1,2000,NTH,WCE,111.0,154.0,False
3,Sat 11 Mar,Domain Stadium,19777.0,1,2000,FRE,GEE,107.0,129.0,False
4,Sat 11 Mar,AAMI Stadium,37222.0,1,2000,ADE,WBD,108.0,131.0,False
...,...,...,...,...,...,...,...,...,...,...
4262,Fri 25 Aug 12:00pm,GMHBA Stadium,NaN,24,2023,GEE,WBD,NaN,NaN,False
4263,Fri 25 Aug 12:00pm,MCG,NaN,24,2023,ESS,COL,NaN,NaN,False
4264,Fri 25 Aug 12:00pm,Marvel Stadium,NaN,24,2023,CAR,GWS,NaN,NaN,False
4265,Fri 25 Aug 12:00pm,Gabba,NaN,24,2023,BRL,STK,NaN,NaN,False


In [57]:
round_ladder['s_round'] = round_ladder['round'] + 1

In [58]:
r_ladder = round_ladder[['year','pf','pa','team','game played','per','s_round','position']]

In [59]:
main = main.merge(r_ladder, how = 'left', left_on = ['year', 'Home', 'round'], right_on = ['year', 'team', 's_round'])
main = main.merge(r_ladder, how = 'left', left_on = ['year', 'Away', 'round'], right_on = ['year', 'team', 's_round'], suffixes=['_home','_away'])

In [60]:
main.columns

Index(['Date', 'Venue', 'Crowd', 'round', 'year', 'Home', 'Away', 'Home Score',
       'Away Score', 'target', 'pf_home', 'pa_home', 'team_home',
       'game played_home', 'per_home', 's_round_home', 'position_home',
       'pf_away', 'pa_away', 'team_away', 'game played_away', 'per_away',
       's_round_away', 'position_away'],
      dtype='object')

In [61]:
main = main.drop(columns=['team_home','team_away'])

In [62]:
main = main.merge(l5, how = 'left', left_on = ['year', 'Home', 'round'], right_on = ['year', 'team', 'round'])
main = main.merge(l5, how = 'left', left_on = ['year', 'Away', 'round'], right_on = ['year', 'team', 'round'], suffixes=['_home_l5','_away_l5'])

In [63]:
main = main.drop(columns=['team_home_l5','team_away_l5'])

In [64]:
main = main.fillna(0)

In [65]:
main.to_csv('/Users/caseyh/Desktop/footyfootyfooty/data_wrangling/round_data.csv',index=False)

In [67]:
main.tail(20)

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,target,...,s_round_away,position_away,result_home_l5,pf_home_l5,pa_home_l5,game played_home_l5,result_away_l5,pf_away_l5,pa_away_l5,game played_away_l5
4247,Sun 13 Aug 3:20pm,Marvel Stadium,0.0,22,2023,STK,RCH,0.0,0.0,False,...,22.0,11.0,8.0,58.0,79.0,5.0,12.0,88.0,75.0,5.0
4248,Sun 13 Aug 4:10pm,Adelaide Oval,0.0,22,2023,POR,GWS,0.0,0.0,False,...,22.0,10.0,12.0,106.0,73.0,5.0,12.0,85.0,72.0,5.0
4249,Fri 18 Aug 7:50pm,Marvel Stadium,0.0,23,2023,COL,BRL,0.0,0.0,False,...,23.0,3.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4250,Sat 19 Aug 1:45pm,MCG,0.0,23,2023,RCH,NTH,0.0,0.0,False,...,23.0,17.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4251,Sat 19 Aug 2:10pm,Heritage Bank Stadium,0.0,23,2023,GCS,CAR,0.0,0.0,False,...,23.0,11.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4252,Sat 19 Aug 4:35pm,GIANTS Stadium,0.0,23,2023,GWS,ESS,0.0,0.0,False,...,23.0,6.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4253,Sat 19 Aug 7:10pm,Adelaide Oval,0.0,23,2023,ADE,SYD,0.0,0.0,False,...,23.0,15.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4254,Sat 19 Aug 7:25pm,Marvel Stadium,0.0,23,2023,STK,GEE,0.0,0.0,False,...,23.0,8.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4255,Sun 20 Aug 1:10pm,Marvel Stadium,0.0,23,2023,WBD,WCE,0.0,0.0,False,...,23.0,18.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
4256,Sun 20 Aug 2:40pm,Optus Stadium,0.0,23,2023,FRE,POR,0.0,0.0,False,...,23.0,2.0,10.0,0.0,0.0,5.0,10.0,0.0,0.0,5.0
